In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=8a8f89b5036f9165398cb1ac154315175f7c04452f8e94984c7fa8e381c71560
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, col,collect_list
from pyspark.sql.window import Window
import pyspark.sql.functions as F
spark=SparkSession.builder.appName('prjoect').getOrCreate()

In [3]:
df=spark.read.option('inferschema','true').option('header','true').csv('/content/2020_olympics_data.csv')

In [ ]:
df.printSchema()

In [17]:
df.show()

+------------------+------+----+-----------+---+----+-----+-------------------+--------------------+-----+
|              Name|Gender| Age|    Country|NOC|Year| City|              Sport|               Event|Medal|
+------------------+------+----+-----------+---+----+-----+-------------------+--------------------+-----+
|   AALERUD Katrine|     F|26.0|     Norway|NOR|2020|Tokyo|       Cycling Road|   Women's Road Race| None|
|   AALERUD Katrine|     F|26.0|     Norway|NOR|2020|Tokyo|       Cycling Road|Women's Individua...| None|
|       ABAD Nestor|     M|28.0|      Spain|ESP|2020|Tokyo|Artistic Gymnastics|    Men's All-Around| None|
|       ABAD Nestor|     M|28.0|      Spain|ESP|2020|Tokyo|Artistic Gymnastics|Men's Floor Exercise| None|
|       ABAD Nestor|     M|28.0|      Spain|ESP|2020|Tokyo|Artistic Gymnastics|  Men's Pommel Horse| None|
|       ABAD Nestor|     M|28.0|      Spain|ESP|2020|Tokyo|Artistic Gymnastics|         Men's Rings| None|
|       ABAD Nestor|     M|28.0|     

# 1. DataFrame showing Medal Count for each country

In [16]:
df.groupBy('Country').agg(
    count(when(df['Medal']=='Gold',True)).alias('Gold'),
    count(when(df['Medal']=='Silver',True)).alias('Silver'),
    count(when(df['Medal']=='Bronze',True)).alias('Bronze'),
    count(when(df['Medal']!='None',True)).alias('Total')).orderBy('Total',ascending=False).show()

+-------------+----+------+------+-----+
|      Country|Gold|Silver|Bronze|Total|
+-------------+----+------+------+-----+
|United States| 113|   110|    75|  298|
|       Russia|  41|    80|    28|  149|
|       France|  65|    42|    34|  141|
|Great Britain|  41|    43|    57|  141|
|        China|  59|    49|    33|  141|
|        Japan|  68|    41|    22|  131|
|    Australia|  36|    28|    67|  131|
|       Canada|  36|    11|    38|   85|
|      Germany|  21|    26|    34|   81|
|        Italy|  22|    17|    37|   76|
|  Netherlands|  33|    22|    17|   72|
|        Spain|   4|    44|    22|   70|
|  New Zealand|  29|    28|     8|   65|
|       Brazil|  29|    17|     9|   55|
|      Hungary|   9|     7|    35|   51|
|    Argentina|   0|    18|    25|   43|
|  South Korea|  14|     7|    17|   38|
|       Poland|  14|    14|     8|   36|
|       Serbia|  15|     1|    19|   35|
|       Sweden|   6|    28|     0|   34|
+-------------+----+------+------+-----+
only showing top

# 2.  DataFrames for each sport showing medal counts for each country


In [22]:
unique_sports = [row['Sport'] for row in df.select('Sport').distinct().collect()]

# Create an empty dictionary to store DataFrames for each sport
sport_dataframes = {}

# Iterate through each sport
for sport in unique_sports:
    # Filter the DataFrame for the current sport
    sport_df = df.filter(df["Sport"] == sport)

    # Group the filtered DataFrame by "Sport" and "Country" and calculate medal counts
    result_df = sport_df.groupBy("Sport", "Country").agg(
        count(when(sport_df["Medal"] == "Gold", True)).alias("Gold"),
        count(when(sport_df["Medal"] == "Silver", True)).alias("Silver"),
        count(when(sport_df["Medal"] == "Bronze", True)).alias("Bronze"),
        count(when(df['Medal']!='None',True)).alias('Total')
    )

    # Define a Window specification for ranking within the sport
    window_spec = Window.partitionBy("Sport").orderBy(
        col("Gold").desc(), col("Silver").desc(), col("Bronze").desc(), col("Total").desc()
    )

    # Add a "Rank" column based on medal counts within the sport
    result_df = result_df.withColumn("Rank", F.row_number().over(window_spec))

    # Filter for the top 10 countries in the sport
    result_df = result_df.filter(col("Rank") <= 10)
    result_df.show()
    # Store the result DataFrame in the dictionary using the sport name as the key
    sport_dataframes[sport] = result_df

# Now you have a dictionary 'sport_dataframes' with DataFrames for the top 10 countries in each sport
# You can access each sport's DataFrame using sport_dataframes[sport_name]
# For example, to access the top 10 countries in "Swimming," use sport_dataframes["Swimming"].show()


+------+--------------+----+------+------+-----+----+
| Sport|       Country|Gold|Silver|Bronze|Total|Rank|
+------+--------------+----+------+------+-----+----+
|Tennis|        Russia|   2|     3|     0|    5|   1|
|Tennis|       Croatia|   2|     2|     0|    4|   2|
|Tennis|Czech Republic|   2|     1|     0|    3|   3|
|Tennis|   Switzerland|   1|     2|     0|    3|   4|
|Tennis|       Germany|   1|     0|     0|    1|   5|
|Tennis|   New Zealand|   0|     0|     2|    2|   6|
|Tennis|     Australia|   0|     0|     2|    2|   7|
|Tennis|        Brazil|   0|     0|     2|    2|   8|
|Tennis|         Spain|   0|     0|     1|    1|   9|
|Tennis|       Ukraine|   0|     0|     1|    1|  10|
+------+--------------+----+------+------+-----+----+

+------+-------------+----+------+------+-----+----+
| Sport|      Country|Gold|Silver|Bronze|Total|Rank|
+------+-------------+----+------+------+-----+----+
|Boxing|         Cuba|   4|     0|     1|    5|   1|
|Boxing|Great Britain|   2|    

# 3. DataFrames for each country showing medal counts in each sports they won


In [34]:
# Get the unique list of countries from the DataFrame
unique_countries = [row['Country'] for row in df.select('Country').distinct().collect()]

# Create an empty dictionary to store DataFrames for each country
country_dataframes = {}

# Iterate through each country
for country in unique_countries:
    # Filter the DataFrame for the current country
    country_df = df.filter(df["Country"] == country)

    # Group the filtered DataFrame by "Country" and "Sport" and calculate medal counts
    result_df = country_df.groupBy("Country", "Sport").agg(
        count(when(country_df["Medal"] == "Gold", True)).alias("Gold"),
        count(when(country_df["Medal"] == "Silver", True)).alias("Silver"),
        count(when(country_df["Medal"] == "Bronze", True)).alias("Bronze"),
        count(when(country_df['Medal'] != 'None', True)).alias('Total')
    )

    # Skip countries with zero total medals
    total_medals = result_df.groupBy("Country").agg(F.sum("Total").alias("Total"))
    if total_medals.filter(col("Total") > 0).count() > 0:
        # Filter out sports with zero medals
        result_df = result_df.filter(col('Total') > 0)

        # Define a Window specification for ranking within the country
        window_spec = Window.partitionBy("Country").orderBy(
            col("Gold").desc(), col("Silver").desc(), col("Bronze").desc(), col("Total").desc()
        )

        # Add a "Rank" column based on medal counts within the country
        result_df = result_df.withColumn("Rank", F.row_number().over(window_spec))
        result_df.show()
        # Store the result DataFrame in the dictionary using the country name as the key
        country_dataframes[country] = result_df

# For example, to access the top sports for "Colombia," use country_dataframes["Colombia"].show()


+-------+-------------------+----+------+------+-----+----+
|Country|              Sport|Gold|Silver|Bronze|Total|Rank|
+-------+-------------------+----+------+------+-----+----+
| Russia|  Artistic Swimming|  10|     0|     0|   10|   1|
| Russia|            Fencing|   9|    10|     1|   20|   2|
| Russia|Artistic Gymnastics|   8|     2|     4|   14|   3|
| Russia|          Wrestling|   4|     0|     4|    8|   4|
| Russia|           Swimming|   2|     7|     1|   10|   5|
| Russia|           Shooting|   2|     5|     3|   10|   6|
| Russia|             Tennis|   2|     3|     0|    5|   7|
| Russia|          Taekwondo|   2|     1|     1|    4|   8|
| Russia|             Boxing|   1|     1|     4|    6|   9|
| Russia|          Athletics|   1|     1|     0|    2|  10|
| Russia|           Handball|   0|    15|     0|   15|  11|
| Russia|         Volleyball|   0|    12|     0|   12|  12|
| Russia|     3x3 Basketball|   0|     8|     0|    8|  13|
| Russia|Rhythmic Gymnastics|   0|     6

# 4. Dataframe showing medal counts of each atheletes and their ranks


In [45]:
# Group the DataFrame by "Name" (player name) and calculate medal counts
result_df = df.groupBy("Name").agg(
    count(when(df["Medal"] == "Gold", True)).alias("Gold"),
    count(when(df["Medal"] == "Silver", True)).alias("Silver"),
    count(when(df["Medal"] == "Bronze", True)).alias("Bronze"),
    count(when(df['Medal'] != 'None', True)).alias('Total'),
    max('Country').alias('Country'),
    max('Gender').alias('Gender'),
    collect_list(when(df['Medal'] != 'None', df['Event'])).alias("Sport Events")  # Collect sports with medals in an array
)

# Filter out players with a total medal count of 0
result_df = result_df.filter(col('Total') > 0)

# Define a Window specification for ranking based on total medal counts
window_spec = Window.orderBy(
    col("Gold").desc(), col("Silver").desc(), col("Bronze").desc(), col("Total").desc()
)

# Add a "Rank" column based on total medal counts
result_df = result_df.withColumn("Rank", F.row_number().over(window_spec)).select('Rank','Name','Gender','Country','Gold','Silver','Bronze','Total','Sport Events')

# Show the result DataFrame
result_df.show(truncate=False)


+----+----------------------+------+-------------+----+------+------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Rank|Name                  |Gender|Country      |Gold|Silver|Bronze|Total|Sport Events                                                                                                                                                                                                               |
+----+----------------------+------+-------------+----+------+------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1   |DRESSEL Caeleb        |M     |United States|5   |0     |0     |5    |[Men's 50m Freestyle, Men's 100m Free

# 5. Dataframe showing Medal counts of  Top atheletes from each country who have won medals


In [57]:
# Group the DataFrame by "Name" (player name) and calculate medal counts
result_df = df.groupBy("Name").agg(
    count(when(df["Medal"] == "Gold", True)).alias("Gold"),
    count(when(df["Medal"] == "Silver", True)).alias("Silver"),
    count(when(df["Medal"] == "Bronze", True)).alias("Bronze"),
    count(when(df['Medal'] != 'None', True)).alias('Total'),
    max('Country').alias('Country'),
    max('Gender').alias('Gender'),
    collect_list(when(df['Medal'] != 'None', df['Event'])).alias("Events")  # Collect events with medals in an array
)

# Filter out players with a total medal count of 0
result_df = result_df.filter(col('Total') > 0)
result_df.filter(col('Total')==0).show()
# Get the unique list of countries from the DataFrame
unique_countries = [row['Country'] for row in result_df.select('Country').distinct().collect()]

# Create a dictionary to store DataFrames for each country
country_dataframes = {}

# Iterate through each country
for country in unique_countries:
    # Filter the DataFrame for the current country
    country_df = result_df.filter(result_df["Country"] == country)


    # Filter for the top 3 athletes in the country
    window_spec = Window.orderBy(
            col("Gold").desc(), col("Silver").desc(), col("Bronze").desc(), col("Total").desc()
        )
    player_df = country_df.withColumn("Rank", F.row_number().over(window_spec))
    top_3_athletes_df = player_df.filter(col("Rank") <= 3).select('Rank','Name','Country','Gender','Gold','Silver','Bronze','Total','Events')
    top_3_athletes_df.show()
    # Store the result DataFrame in the dictionary using the country name as the key
    country_dataframes[country] = top_3_athletes_df



+----+----+------+------+-----+-------+------+------+
|Name|Gold|Silver|Bronze|Total|Country|Gender|Events|
+----+----+------+------+-----+-------+------+------+
+----+----+------+------+-----+-------+------+------+

+----+--------------------+-------+------+----+------+------+-----+--------------------+
|Rank|                Name|Country|Gender|Gold|Silver|Bronze|Total|              Events|
+----+--------------------+-------+------+----+------+------+-----+--------------------+
|   1|BATSARASHKINA Vit...| Russia|     F|   2|     1|     0|    3| Air Pistol W...|
|   2|        RYLOV Evgeny| Russia|     M|   2|     1|     0|    3|[Men's 100m Backs...|
|   3|KOLESNICHENKO Sve...| Russia|     F|   2|     0|     0|    2|   [Duet Team, Team]|
+----+--------------------+-------+------+----+------+------+-----+--------------------+

+----+--------------+--------------+------+----+------+------+-----+--------------------+
|Rank|          Name|       Country|Gender|Gold|Silver|Bronze|Total|     

# Advance Code

In [45]:
# Import necessary Spark libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, col, collect_list, max
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# Create a Spark session
spark = SparkSession.builder.appName('project').getOrCreate()

# Load the dataset
df = spark.read.option('inferschema', 'true').option('header', 'true').csv('/content/2020_olympics_data.csv')

# Function to calculate medal counts for a given group column
def result(df, group_col, partition_col=None, rank_limit=None):
    # Define the aggregation columns for medal counts
    agg_columns = [
        count(when(df['Medal'] == 'Gold', True)).alias('Gold'),
        count(when(df['Medal'] == 'Silver', True)).alias('Silver'),
        count(when(df['Medal'] == 'Bronze', True)).alias('Bronze'),
        count(when(df['Medal'] != 'None', True)).alias('Total'),
    ]

    # Create a window specification for ranking
    window_spec = Window.partitionBy(partition_col).orderBy(
        col("Gold").desc(), col("Silver").desc(), col("Bronze").desc(), col("Total").desc()
    ) if partition_col else Window.orderBy(
        col("Gold").desc(), col("Silver").desc(), col("Bronze").desc(), col("Total").desc()
    )

    # If both group_col and partition_col are provided, aggregate by both columns
    if group_col and partition_col:
        result_df = df.groupBy(partition_col, group_col).agg(*agg_columns).filter(col('Total') > 0)
    # If only group_col is provided, aggregate by that column
    elif group_col:
        result_df = df.groupBy(group_col).agg(*agg_columns).filter(col('Total') > 0)

    # If the group column is "Name," also collect the list of events
    if group_col == "Name":
        events_df = df.filter(df['Medal'] != 'None').groupBy(group_col).agg(
            collect_list("Event").alias('Event')
        )
        result_df = result_df.join(events_df, [group_col], "left")

    # Calculate the rank for each group
    result_df = result_df.withColumn("Rank", F.row_number().over(window_spec))

    # If rank_limit is provided, filter rows by rank
    if rank_limit:
        result_df = result_df.filter(col('Rank') <= rank_limit)

    # Show the result
    result_df.show()
    return result_df

# Perform different analyses using the defined functions
# 1. Medal Count for countries
result_df1=result(df, 'Country')

# 2. Medal Count of Top Countries in Each Sport
result_df2=result(df, 'Country', 'Sport')

# 3. Medal Count of Top Sport Performances in Each Country
result_df3=result(df, 'Sport', 'Country')

# 4. Top  Athletes
result_df4=result(df, 'Name')

# 5. Details of Top Athletes in Each Country along with countries with no medals
result_df5=result(df, 'Name', 'Country')


+-------------+----+------+------+-----+----+
|      Country|Gold|Silver|Bronze|Total|Rank|
+-------------+----+------+------+-----+----+
|United States| 113|   110|    75|  298|   1|
|        Japan|  68|    41|    22|  131|   2|
|       France|  65|    42|    34|  141|   3|
|        China|  59|    49|    33|  141|   4|
|       Russia|  41|    80|    28|  149|   5|
|Great Britain|  41|    43|    57|  141|   6|
|    Australia|  36|    28|    67|  131|   7|
|       Canada|  36|    11|    38|   85|   8|
|  Netherlands|  33|    22|    17|   72|   9|
|  New Zealand|  29|    28|     8|   65|  10|
|       Brazil|  29|    17|     9|   55|  11|
|        Italy|  22|    17|    37|   76|  12|
|      Germany|  21|    26|    34|   81|  13|
|      Belgium|  20|     1|     6|   27|  14|
|       Serbia|  15|     1|    19|   35|  15|
|       Poland|  14|    14|     8|   36|  16|
|  South Korea|  14|     7|    17|   38|  17|
|         Fiji|  13|     0|    13|   26|  18|
|      Hungary|   9|     7|    35|

AttributeError: ignored

import pandas as pd

# Define a function to export multiple DataFrames to an Excel file with different worksheets
def export_to_excel(dataframes, filename):
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        for sheet_name, df in dataframes.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

# Example usage:
# Create a dictionary with sheet names as keys and DataFrames as values
dataframes = {
    'MedalCountsByCountry': result_df1,  # Replace with your actual DataFrames
    'MedalCountsBySport': result_df2,
    'Top10CountriesInEachSport': result_df3,
    'Top10SportPerformanceInEachCountry': result_df4,
    'Top10Athletes': result_df5
}

# Specify the Excel file name
excel_filename = 'olympic_report.xlsx'

# Export the DataFrames to the Excel file with different worksheets
export_to_excel(dataframes, excel_filename)


In [ ]:
!pip install XlsxWriter

In [46]:
import pandas as pd
import xlsxwriter

# Define a function to export multiple DataFrames to an Excel file with different worksheets
def export_to_excel(dataframes, filename):
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        for sheet_name, df in dataframes.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

pandas_df1 = result_df1.toPandas()
print(pandas_df1)
pandas_df2 = result_df2.toPandas()
pandas_df3 = result_df3.toPandas()
pandas_df4 = result_df4.toPandas()
pandas_df5 = result_df5.toPandas()

# Example usage:
# Create a dictionary with sheet names as keys and DataFrames as values
dataframes = {
    'MedalCountsByCountry': pandas_df1,
    'TopCountriesInSport': pandas_df2,  # Modified sheet name
    'TopSportsInEachCountry': pandas_df3,  # Modified sheet name
    'Top10Athletes': pandas_df4,
    'TopAthletesInEachCountry': pandas_df5  # Modified sheet name
}

# Specify the Excel file name
excel_filename = '/content/olympic_report.xlsx'

# Export the DataFrames to the Excel file with different worksheets
export_to_excel(dataframes, excel_filename)


          Country  Gold  Silver  Bronze  Total  Rank
0   United States   113     110      75    298     1
1           Japan    68      41      22    131     2
2          France    65      42      34    141     3
3           China    59      49      33    141     4
4          Russia    41      80      28    149     5
..            ...   ...     ...     ...    ...   ...
88         Kuwait     0       0       1      1    89
89        Grenada     0       0       1      1    90
90          Syria     0       0       1      1    91
91   Burkina Faso     0       0       1      1    92
92        Moldova     0       0       1      1    93

[93 rows x 6 columns]


In [34]:

# Example usage:
# result_df(df, group_col, partition_col, rank_limit)
result_df(df, 'Country')  # Medal Count for countries
result_df(df, 'Country', 'Sport', rank_limit=10)  # Medal Count of Top 10 Countries in Each Sport
result_df(df, 'Sport', 'Country', rank_limit=10)  # Medal Count of Top 10 Sport Performance in Each Country
result_df(df, 'Name', rank_limit=10)  # Top 10 Athletes
result_df(df, 'Name', 'Country', rank_limit=3)  # Details of 3 Athletes in Each Country


+-------------+----+------+------+-----+----+
|      Country|Gold|Silver|Bronze|Total|Rank|
+-------------+----+------+------+-----+----+
|United States| 113|   110|    75|  298|   1|
|        Japan|  68|    41|    22|  131|   2|
|       France|  65|    42|    34|  141|   3|
|        China|  59|    49|    33|  141|   4|
|       Russia|  41|    80|    28|  149|   5|
|Great Britain|  41|    43|    57|  141|   6|
|    Australia|  36|    28|    67|  131|   7|
|       Canada|  36|    11|    38|   85|   8|
|  Netherlands|  33|    22|    17|   72|   9|
|  New Zealand|  29|    28|     8|   65|  10|
|       Brazil|  29|    17|     9|   55|  11|
|        Italy|  22|    17|    37|   76|  12|
|      Germany|  21|    26|    34|   81|  13|
|      Belgium|  20|     1|     6|   27|  14|
|       Serbia|  15|     1|    19|   35|  15|
|       Poland|  14|    14|     8|   36|  16|
|  South Korea|  14|     7|    17|   38|  17|
|         Fiji|  13|     0|    13|   26|  18|
|      Hungary|   9|     7|    35|